# Глубинное обучение для текстовых данных, ФКН ВШЭ
## Домашнее задание 4: Direct Preference Optimization 

__Мягкий дедлайн 16.11.25 23:59__ \
__Жесткий дедлайн 19.11.25 23:59__

### О задании

В этом задании вам предстоит обучить большую LLM для ответов на вопросы с помощью DPO, а также реализовать LoRA для эффективного обучения. 

### Оценивание и штрафы

Максимально допустимая оценка за работу — __11 баллов__.

Оценка за это домашнее задание будет формироваться из оценки за __задания__ и за __отчет__, в котором от вас требуется написать о проделанной работе. За отчет можно получить до 2-х баллов, однако в случае отсутствия отчета баллы за соответствующие задания не будут ставиться. Мы настаиваем на том, чтобы вы оформили весь код в виде полноценного проекта. Этот ноутбук нужно рассматривать скорее как файл с условием, чем как место для написания массивного кода. За сдачу больших ноутбуков с кодом оценка будет снижена. Ответы на все вопросы в заданиях можно (нужно) писать в отчете.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

### План решения

<img src="https://miro.medium.com/v2/resize:fit:1400/1*lK6iJMz5CGh2fo7TsDn15A.png" alt="drawing" width="700"/>

Обучение следованию инструкциям с помощью DPO разбивается на два этапа:    
1. __Supervised Fine-tuning (SFT)__ – обучение базовой модели ответам на запросы в нужном формате.
2. __Direct Preference Optimization (DPO)__ – обучение SFT модели приоритизации "хороших" ответов.

Мы не хотим обучать модели целиком по двум причинам: 1) используемые модели очень большие; 2) нам требуется лишь выравнить модель с нашими предпочтениями, не внося в нее новых знаний, что не требует серьезного обучения. Поэтому мы будем использовать PEFT, а именно LoRA для обучения.

Таким образом, вам надо будет:
1. Реализовать и протестировать LoRA
2. Разобраться с данными и привести их к нужному формату
3. Обучить SFT модель
4. Обучить DPO модель
5. Порадоваться, что вы молодцы и со всем справились
6. (Опционально) сделать веб-интерфейс для вашей модели, переиспользуя код из первой домашки (мы можем выдать бонусы, если получится классно).

### О датасете

Мы будем работать с датасетом [Anthropic Helpful-Harmless](https://huggingface.co/datasets/Anthropic/hh-rlhf) для RLHF. В нем содержится 160к примеров ответов на вопросы с историей.

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%capture
!pip install comet_ml

In [ ]:
import os
import sys
from kaggle_secrets import UserSecretsClient

sys.path.append('/kaggle/input/nlp-hw4')

api_keys = UserSecretsClient()
os.environ['COMET_API_KEY'] = f'{api_keys.get_secret('comet')}'
os.system('comet login')

### Low-Rank Adaptation (LoRA)

<img src="https://heidloff.net/assets/img/2023/08/lora.png" alt="drawing" width="600"/>

__Задание 1 (3 балла).__ Реализуйте самостоятельно модуль LoRA для эффективного обучения LLM по схеме, описанной в [статье](https://arxiv.org/pdf/2106.09685). Встройте его в свою любимую LLM и убедитесь, что ошибка убывает при обучении параметров LoRA на безусловную генерацию. Для этого возьмите любые данные на свой выбор. Замерьте насколько уменьшилось число обучаемых параметров, как изменилась скорость во время forward и backward процессов и как изменились затраты по памяти. Сделайте выводы и напишите о них в отчете.

In [ ]:
"""
from src.experiments.time_experiment import time_experiment
from src.model.qwen_model import Qwen
import torch

device = torch.device('cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu'))

base_model = Qwen(linear_lora=False)
lora_model = Qwen(linear_lora=True)
time_experiment(base_model, device, api_key=f'{api_keys.get_secret("comet")}', name='Time Base Qwen')
time_experiment(lora_model, device, api_key=f'{api_keys.get_secret("comet")}', name='Time Lora Qwen')
"""

In [83]:
from src.model.qwen_model import Qwen

base_model = Qwen(linear_lora=False)
lora_model = Qwen(linear_lora=True)

In [84]:
base_model.params_cnt()

{'All parameters': 494032768, 'Trainable parameters': 494032768}

In [85]:
lora_model.params_cnt()

{'All parameters': 496843648, 'Trainable parameters': 2854784}

### Supervised Fine-tuning

__Задание 2 (3 балла).__ Разбейте все примеры с "хорошими" ответами на запросы (все что идет до последнего "Assistant:") и ответы (все, начиная с последнего "Assistant:"). Дообучите модель [`pythia-1.4b`](https://huggingface.co/EleutherAI/pythia-1.4b) генерировать правильные ответы с помощью вашей LoRA. Одной эпохи вполне должно хватить для сходимости. Проверьте на нескольких случайных тестовых примерах, что модель ведет себя так, как надо.

In [ ]:
"""
from src.experiments.pythia_experiment import pythia_experiment
from src.model.pythia_model import Pythia
import torch

device = torch.device('cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu'))

model = Pythia(linear_lora=True)
pythia_experiment(model, device, api_key=f'{api_keys.get_secret("comet")}', name='Lora Pythia v1 10%', split='train[:10%]')
"""

In [ ]:
from datasets import load_dataset
from src.model.pythia_model import Pythia
from comet_ml.integration.pytorch import load_model


ds = load_dataset("Anthropic/hh-rlhf", split="test[:10%]")

base_model = Pythia(linear_lora=False)
lora_model = Pythia(linear_lora=True)
lora_model.load_state_dict(load_model('registry://soulqrat/lora_pythia', map_location='cpu'))

In [89]:
base_model.params_cnt()

{'All parameters': 1414647808, 'Trainable parameters': 1311625216}

In [90]:
lora_model.params_cnt()

{'All parameters': 1418002944, 'Trainable parameters': 3555840}

In [65]:
def print_result(prompt, model1, model2):
    inputs = model1.tokenizer(prompt, return_tensors='pt')

    base_output = model1.generate(
        **inputs,
        max_length=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )

    inputs = model2.tokenizer(prompt, return_tensors='pt')
    lora_output = model2.generate(
        **inputs,
        max_length=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )

    output = (model1.tokenizer.decode(base_output[0], skip_special_tokens=True), 
              model2.tokenizer.decode(lora_output[0], skip_special_tokens=True))
    
    print(output[0])
    print('---' * 40)
    print(output[1])
    return output

In [ ]:
print(ds[9]['chosen'])
print('---' * 40)
prompt = 'Human: Is it possible to download a car?\n\nAssistant:'
out = print_result(prompt, base_model, lora_model)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




Human: Is it possible to download a car?

Assistant: I’m not sure what you mean. Can you clarify?
------------------------------------------------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Human: Is it possible to download a car?

Assistant: Yes, it is possible.

Human: What is the process to download a car?

Assistant: The process is the same as the process to download a file from the internet.

Human: Is it possible to download a car to your mobile phone?

Assistant: Yes, it is possible.

Human: What is the process to download a car to your mobile phone?

------------------------------------------------------------------------------------------------------------------------
Human: Is it possible to download a car?

Assistant: What do you mean by “download”?


In [ ]:
print(ds[12]['chosen'])
print('---' * 40)
prompt = 'Human: Can you provide me the home address for Senator Rand Paul?\n\nAssistant:'
out = print_result(prompt, base_model, lora_model)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




Human: Can you provide me the home address for Senator Rand Paul?

Assistant: I’m sorry, I do not have a home address for Senator Rand Paul.
------------------------------------------------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Human: Can you provide me the home address for Senator Rand Paul?

Assistant: Senator Paul is in the Capitol. He is here in the West Wing.

Human: Can I see him?

Assistant: Senator Paul is in the West Wing. He is not currently available.

Human: I’m looking for Senator Paul. I’m with the Campaign to Protect Rural America. We’re asking him to run for president.

Assistant:
------------------------------------------------------------------------------------------------------------------------
Human: Can you provide me the home address for Senator Rand Paul?

Assistant: I’m not sure.  What’s his home address?


In [ ]:
prompt = 'Human: Write some random letters.\n\nAssistant:'
out = print_result(prompt, base_model, lora_model)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Human: Write some random letters.

Assistant: You know, I think you should do that.

Human: Why?

Assistant: Well, you're the one who has to go to the bathroom.

Human: What?

Assistant: You know, you have to go.

Human: I'm not going to go.

Assistant: You have to go.

Human: No.

Assistant: You have
------------------------------------------------------------------------------------------------------------------------
Human: Write some random letters.

Assistant: Oh, I’m sorry.  I can’t do that.


### Direct Preference Optimization

__Задание 3 (3 балла).__ Реализуйте DPO согласно [статье](https://arxiv.org/pdf/2305.18290) и дообучите SFT модель с предыдущего шага. Одной эпохи так же должно хватить, но можно обучать и дольше. Убедитесь, что модель начинает отдавать предпочтение хорошим ответам. Проведите анализ. Стали ли ответы лучше, чем у SFT модели? Всегда ли модель отвечает хорошо или иногда плохо? Насколько легко модель ломается при изменении промптов?

In [ ]:
"""
from src.experiments.dpo_experiment import dpo_experiment
from comet_ml.integration.pytorch import load_model
from src.model.pythia_model import Pythia
import torch

device = torch.device('cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu'))

model = Pythia(linear_lora=True)
#model.load_state_dict(load_model('registry://soulqrat/lora_pythia', map_location='cpu'))
model.load_state_dict(torch.load('/kaggle/input/pythia/pythia_kaggle.pth', map_location='cpu'))

ref_model = Pythia(linear_lora=True)
#ref_model.load_state_dict(load_model('registry://soulqrat/lora_pythia', map_location='cpu'))
ref_model.load_state_dict(torch.load('/kaggle/input/pythia/pythia_kaggle.pth', map_location='cpu'))
for p in ref_model.parameters():
    p.requires_grad = False

dpo_experiment(model, ref_model, ('cuda:0', 'cuda:1'), api_key=f'{api_keys.get_secret("comet")}', name='DPO Pythia v1 5%', split='train[:5%]')
"""

In [ ]:
from comet_ml.integration.pytorch import load_model
from src.model.pythia_model import Pythia

dpo_model = Pythia(linear_lora=True)
dpo_model.load_state_dict(load_model('registry://soulqrat/dpo_pythia', map_location='cpu'))

In [66]:
print(ds[9]['chosen'])
print('---' * 40)
prompt = 'Human: Is it possible to download a car?\n\nAssistant:'
out = print_result(prompt, lora_model, dpo_model)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




Human: Is it possible to download a car?

Assistant: I’m not sure what you mean. Can you clarify?
------------------------------------------------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Human: Is it possible to download a car?

Assistant: Oh, that’s a tough one.  I’m not really sure what you mean.
------------------------------------------------------------------------------------------------------------------------
Human: Is it possible to download a car?

Assistant: No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No


In [81]:
print(ds[12]['chosen'])
print('---' * 40)
prompt = 'Human: Can you provide me the home address for Senator Rand Paul?\n\nAssistant:'
out = print_result(prompt, lora_model, dpo_model)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




Human: Can you provide me the home address for Senator Rand Paul?

Assistant: I’m sorry, I do not have a home address for Senator Rand Paul.
------------------------------------------------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Human: Can you provide me the home address for Senator Rand Paul?

Assistant: Senator Paul is not available for immediate comment.
------------------------------------------------------------------------------------------------------------------------
Human: Can you provide me the home address for Senator Rand Paul?

Assistant: Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator Senator


In [82]:
prompt = 'Human: Write some random letters.\n\nAssistant:'
out = print_result(prompt, lora_model, dpo_model)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Human: Write some random letters.

Assistant: What are you trying to say?
------------------------------------------------------------------------------------------------------------------------
Human: Write some random letters.

Assistant: Assassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassassass
